In [12]:
# Python natives
import os

os.chdir("/home/tim/Development/OCPPM/")
import pprint
import lightgbm as lgb
import numpy as np
import pandas as pd
import sklearn.metrics as metrics
from sklearn.model_selection import train_test_split
from utilities import evaluation_utils

In [2]:
object_attributes = ["oa1", "oa5", "oa10"]
oft_out_file = "data/CS/feature_encodings/baselines/OFT/objects.csv"

In [3]:
df_objects = pd.read_csv(oft_out_file, sep=";")
df_objects = df_objects[object_attributes + ["ea4"]]

In [8]:
df_objects["ea4"].value_counts(normalize=True).sort_index()

0.0    0.706659
1.0    0.012044
2.0    0.008753
3.0    0.014572
4.0    0.084501
5.0    0.073626
6.0    0.099845
Name: ea4, dtype: float64

In [9]:
# make train test split
X, y = (
    df_objects.drop(columns=["ea4"]),
    df_objects["ea4"],
)
X_train, X_valid, y_train, y_valid = train_test_split(
    X, y, test_size=0.3, random_state=42
)

train_data = lgb.Dataset(X_train, label=y_train)
valid_data = lgb.Dataset(X_valid, label=y_valid)

In [ ]:
params = {
    "num_threads": 4,
    "objective": "multiclass",
    "num_class": 7,
    "metric": ["multi_logloss"],
    "num_boost_round": 100,
    "stopping_rounds": 100,
}

bst = lgb.train(
    params,
    train_data,
    valid_sets=[valid_data],
    callbacks=[lgb.early_stopping(params["stopping_rounds"])],
)

In [11]:
y_train_probs = bst.predict(X_train)
y_valid_probs = bst.predict(X_valid)

y_train_preds = np.apply_along_axis(
    evaluation_utils.get_preds_from_probs, axis=1, arr=y_train_probs
)
y_valid_preds = np.apply_along_axis(
    evaluation_utils.get_preds_from_probs, axis=1, arr=y_valid_probs
)

In [13]:
# Run model evaluation, TODO: store this somewhere (in JSON)
eval_train = evaluation_utils.get_evaluation(
    y_train, y_train_preds, classification=True
)
eval_valid = evaluation_utils.get_evaluation(
    y_valid, y_valid_preds, classification=True
)
experiment_settings = {"experiment_settings": params}
evaluation_report = {"train": eval_train, "validation": eval_valid}
pprint.pprint(evaluation_report)

/home/tim/Development/OCPPM/.env/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/tim/Development/OCPPM/.env/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/tim/Development/OCPPM/.env/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


{'train': {'report': {'0.0': {'f1-score': 0.8389990876567128,
                              'precision': 0.7319553700146311,
                              'recall': 0.9827147330605345,
                              'support': 208212},
                      '1.0': {'f1-score': 0.0,
                              'precision': 0.0,
                              'recall': 0.0,
                              'support': 3515},
                      '2.0': {'f1-score': 0.0,
                              'precision': 0.0,
                              'recall': 0.0,
                              'support': 2584},
                      '3.0': {'f1-score': 0.0,
                              'precision': 0.0,
                              'recall': 0.0,
                              'support': 4339},
                      '4.0': {'f1-score': 0.0008053799379857448,
                              'precision': 0.4166666666666667,
                              'recall': 0.0004030795275907937,
          

/home/tim/Development/OCPPM/.env/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/tim/Development/OCPPM/.env/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/tim/Development/OCPPM/.env/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
